<a href="https://colab.research.google.com/github/hanumantjain/agentic-ai/blob/main/Building_a_Quasi_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!!pip install litellm --quiet

import os
from google.colab import userdata
api_key = userdata.get('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = api_key

In [8]:
from litellm import completion
from typing import List, Dict

def generate_response(messages: List[Dict]) -> str:
  response= completion(
      model='openai/gpt-4o-mini',
      messages= messages,
      max_tokens=512
  )
  return response.choices[0].message.content
what_function_to_create = input('What function to create: ')
messages=[
    {'role':'system', 'content': 'You are an expert software engineer who uses python that generates functions and gives separate code from commentry'},
    {'role':'user', 'content':what_function_to_create},
]

response = generate_response(messages)
print(response)

messages.append({'role':'assistant', "content":response})
messages.append({'role':'user', 'content':'Add comprehensive documentation including function description, parameter description, return value, example and edge cases'})

response = generate_response(messages)
print(response)

messages.append({'role':'assistant', 'content':response})
messages.append({'role':'user', 'content':'Add the test cases using Python unittest framework which includes basic functionality, edge cases, error cases and various input scenarios'})

response = generate_response(messages)
print(response)

What function to create: check prime number
Here's a Python function to check if a number is prime:

```python
def is_prime(n):
    """Check if a number is prime."""
    if n <= 1:
        return False
    if n <= 3:
        return True
    if n % 2 == 0 or n % 3 == 0:
        return False
    i = 5
    while i * i <= n:
        if n % i == 0 or n % (i + 2) == 0:
            return False
        i += 6
    return True
```

### Commentary
1. **Function Definition**: The function `is_prime` takes a single argument `n`, which is the number to be checked.
2. **Basic Checks**: 
   - If `n` is less than or equal to 1, it returns `False` since such numbers are not prime.
   - If `n` is 2 or 3, it returns `True` as both are prime.
3. **Divisibility Checks**: 
   - It checks if `n` is divisible by 2 or 3 and returns `False` if so.
4. **Loop for Larger Numbers**: 
   - It uses a loop starting from 5 and checks divisibility with `i` and `i + 2` (i.e., checks 5, 7, 11, 13, ...), continuing until `

In [11]:
def generate_response(messages: List[Dict]) -> str:
  response = completion(
      model='openai/gpt-4o-mini',
      messages = messages,
      max_tokens=512
  )
  return response.choices[0].message.content

def extract_code_block(response: str) -> str:
  if not '```' in response:
    return response
  code_block = response.split('```')[1].strip()

  if code_block.startswith('python'):
    code_block = code_block[6:]

  return code_block

def develop_custom_function():
  print('\nWhat kind of function would you like to create?')
  print('\nYour Descirption: ', end='')
  function_description = input().strip()

  messages = [
      {'role':'system', 'content':'You are a Python expert helping to develop a function.'}
  ]

  messages.append({
      'role': 'user', 'content':f'Write a function that {function_description}. Output the function in a python code block.'
  })

  initital_function = generate_response(messages)

  initital_function = extract_code_block(initital_function)

  print('\n==== Initital Function ====')
  print(initital_function)

  messages.append({
      'role':'assistant', 'content':'python\n\n+initiatal_function+\n\n'
  })

  #second prompt

  messages.append({
      'role':'user', 'content':'Add a comprehensive documentation to this function, incliding description, parameters, return value, examples and edge cases. Output the function in a ```python code block```'
  })

  documented_function = generate_response(messages)
  documented_function = extract_code_block(documented_function)
  print('\n==== Documented Function ====')
  print(documented_function)

  messages.append({
      'role': 'assistant', 'content':'python\n\n+documented_function+\n\n'
  })

  #third prompt

  messages.append({
      'role':'user', 'content':'Add unittest test cases for this function, including tests for basic functionality, edge cases, error cases, and various input scenarios. Output the code in a python code block.'
  })

  test_cases = generate_response(messages)
  test_cases = extract_code_block(test_cases)
  print('\n==== Test Cases ====')
  print(test_cases)

  #generate file name from fucntion description
  filename = function_description.lower()
  filename = ''.join(c for c in filename if c.isalnum() or c.isspace())
  filename = filename.replace(' ','_')[:30] + '.py'

  #save final version
  with open(filename, 'w') as f:
    f.write(documented_function +'\n\n' + test_cases)

  return documented_function, test_cases, filename

if __name__ == "__main__":

  function_code, tests, filename = develop_custom_function()
  print(f'\nFinal code has been saved to {filename}')


What kind of function would you like to create?

Your Descirption: check prime number

==== Initital Function ====

def is_prime(n):
    """Check if a number is a prime number.
    
    Args:
        n (int): The number to check.
        
    Returns:
        bool: True if n is prime, False otherwise.
    """
    if n <= 1:
        return False
    if n <= 3:
        return True
    if n % 2 == 0 or n % 3 == 0:
        return False
    
    i = 5
    while i * i <= n:
        if n % i == 0 or n % (i + 2) == 0:
            return False
        i += 6
    return True

==== Documented Function ====

def is_prime(n):
    """
    Check if a number is a prime number.

    A prime number is a natural number greater than 1 that has no positive divisors other than 1 and itself.

    Parameters:
    n (int): The number to check for primality. Must be a positive integer.

    Returns:
    bool: True if n is a prime number, False otherwise.

    Examples:
    >>> is_prime(2)
    True
    >>> is_p